In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import scipy.stats as st
from scipy.stats import linregress

In [4]:
# Output File (CSV)
#output_data_file = "output_data/fire_ca.csv"

##switch to using this file, which has dropped columns

In [5]:
#load in file
fire_data_file = "Resources/California_Fire_Incidents.csv"

#read and display csv with Pandas 
fire_data_df = pd.read_csv(fire_data_file)
fire_data_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Started,Time.1,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,8/17/13,15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,5/30/13,15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,7/15/13,13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,8/10/13,16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,5/2/13,07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [6]:
#list all of the columns in the table
fire_data_df.columns

Index(['AcresBurned', 'Active', 'AdminUnit', 'AirTankers', 'ArchiveYear',
       'CalFireIncident', 'CanonicalUrl', 'ConditionStatement',
       'ControlStatement', 'Counties', 'CountyIds', 'CrewsInvolved', 'Dozers',
       'Engines', 'Extinguished', 'Time', 'Days Burned', 'Fatalities',
       'Featured', 'Final', 'FuelType', 'Helicopters', 'Injuries', 'Latitude',
       'Location', 'Longitude', 'MajorIncident', 'Name', 'PercentContained',
       'PersonnelInvolved', 'Public', 'SearchDescription', 'SearchKeywords',
       'Started', 'Time.1', 'Status', 'StructuresDamaged',
       'StructuresDestroyed', 'StructuresEvacuated', 'StructuresThreatened',
       'UniqueId', 'Updated', 'WaterTenders'],
      dtype='object')

In [29]:
county_count = fire_data_df["Counties"].value_counts()
county_count
# county_df = pd.DataFrame(county_count)
# county_df

Riverside          146
San Diego           89
Butte               66
San Luis Obispo     64
Shasta              64
Kern                62
Fresno              57
Siskiyou            57
San Bernardino      53
Tehama              51
Lake                49
Los Angeles         46
Monterey            45
Santa Clara         39
El Dorado           37
Lassen              36
Madera              36
Tulare              35
Mariposa            35
Alameda             32
Modoc               31
Ventura             30
Santa Barbara       29
Mendocino           28
Contra Costa        27
Napa                25
Humboldt            25
Tuolumne            22
Calaveras           22
Trinity             21
San Benito          20
Stanislaus          20
Solano              19
Sonoma              18
Nevada              17
Placer              17
Merced              16
Yuba                14
Amador              13
Yolo                12
Inyo                12
Plumas              11
Sacramento          11
Mono       

In [34]:
#list unique counties

county_names = fire_data_df["Counties"].unique()
county_names

array(['Tuolumne', 'Los Angeles', 'Riverside', 'Placer', 'Ventura',
       'Fresno', 'Siskiyou', 'Humboldt', 'Tehama', 'Shasta', 'San Diego',
       'Kern', 'Sonoma', 'Contra Costa', 'Butte', 'Tulare',
       'Santa Barbara', 'Mariposa', 'Monterey', 'El Dorado',
       'San Bernardino', 'Plumas', 'Modoc', 'San Luis Obispo', 'Madera',
       'Inyo', 'Napa', 'San Benito', 'San Joaquin', 'Lake', 'Alameda',
       'Glenn', 'Yolo', 'Sacramento', 'Stanislaus', 'Solano', 'Merced',
       'Mendocino', 'Lassen', 'Amador', 'Yuba', 'Nevada', 'Santa Clara',
       'Calaveras', 'San Mateo', 'Orange', 'Colusa', 'Trinity',
       'Del Norte', 'Mono', 'Alpine', 'Sutter', 'Kings', 'Sierra',
       'Santa Cruz', 'Marin', 'Mexico', 'State of Oregon',
       'State of Nevada'], dtype=object)

In [24]:
top_ten_counties = county_df.head(10).reset_index()
top_ten_counties

,index,Counties
0,Riverside,146
1,San Diego,89
2,Butte,66
3,San Luis Obispo,64
4,Shasta,64
5,Kern,62
6,Fresno,57
7,Siskiyou,57
8,San Bernardino,53
9,Tehama,51


In [42]:
#find count of fires deemed a major incident in each county, using "MajorIncident" column

#major_incident_count = 

major_incident_count = fire_data_df.groupby("Counties")["MajorIncident"].sum().reset_index().sort_values(by= "MajorIncident", ascending = False)
major_incident_count

#add in county_count to compare total fires vs. major incidents

# major_all_df = pd.DataFrame({"Counties" : county_names, 
#                "Total Fires" : county_count, 
#                "Major Fires" : major_incident_count})
# major_all_df

,Counties,MajorIncident
3,Butte,28.0
36,San Diego,27.0
43,Shasta,26.0
15,Lake,21.0
32,Riverside,18.0
27,Napa,16.0
8,El Dorado,15.0
20,Mariposa,14.0
9,Fresno,12.0
52,Tehama,11.0


In [40]:
#chart top five counties with 12 or more major fires
major_incident_count.loc[(major_incident_count["MajorIncident"]>=12), ["Counties", "MajorIncident"]]


,Counties,MajorIncident
3,Butte,28.0
36,San Diego,27.0
43,Shasta,26.0
15,Lake,21.0
32,Riverside,18.0
27,Napa,16.0
8,El Dorado,15.0
20,Mariposa,14.0
9,Fresno,12.0
